
how to scan sqlite tools?





how to scan chat logs?




In [ ]:

//import {fileURLToPath} from "url";
const fs = require('fs')
const path = require('path')
const importNotebook = importer.import
const {askLlamaAboutEmotions} = importNotebook('ask llm about emotions')
const {storeChatHistory} = importNotebook('how to cache chat logs')
const { chatCache } = importer.import('cache chat history with llm descriptions')
const { askLlamaAboutConversation, askLlamaAboutCategory } = importNotebook('ask llm about chat conversations')

const hashCode = s => s.split('').reduce((a,b) => (((a << 5) - a) + b.charCodeAt(0))|0, 0)

const CHAT_DIRECTORIES = [
  '/Volumes/External/Personal/Collections/conversations',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/AIM/Query',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/MSN/Query',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/old logs/AIM',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/old logs/MSN',
]

const CHAT_DATES = [
  /(January|February|March|April|May|June|July|August|September|October|November|December) (\d{1,2}), \d{4}/,
  /(0[1-9]|[12]\d|3[01])\.(0[1-9]|1[0-2])\.\d{4}/,
  /(0[1-9]|1[0-2])\/(0[1-9]|[12]\d|3[01])\/\d{4}/,
  /\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])/,
  /(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d{1,2}), \d{4}/
]

const CHAT_TIMES = [
  /(0[1-9]|1[0-2]):([0-5]\d):([0-5]\d)\s?(AM|PM)/i,
  /([01]\d|2[0-3]):([0-5]\d):([0-5]\d)/,
  /(0[1-9]|1[0-2]):([0-5]\d)\s?(AM|PM)/i,
  /([01]\d|2[0-3]):([0-5]\d)/
]

async function askLlamaAboutChatLog(chatLog) {
  if(!chatLog) {
    chatLog = 'anti_milestone@hotmail.com'
  }

  let chatHistory = []
  let chatPath
  for(let i = 0; i < CHAT_DIRECTORIES.length; i++) {
    chatPath = path.join(CHAT_DIRECTORIES[i], chatLog + '.log')
    if(fs.existsSync(chatPath)) {
      chatHistory = fs.readFileSync(chatPath).toLocaleString('utf-8').split('\n')
      break
    }
  }

  let mtime = fs.statSync(chatPath).mtime.getTime()
  let cellCounter = 0
  let currentMessages = []
  let currentTotal = 0
  let cellId = chatPath + '[' + cellCounter + ']'
  for(let i = 0; i < chatHistory.length; i++) {
    if(currentMessages.length > 0 && currentTotal + chatHistory[i].length > 2048 || currentMessages.length == 20) {
      // process now
      // TODO: store result
      let messageBlock = currentMessages.join('\n')
      let date = CHAT_DATES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
      let time = CHAT_TIMES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
      let now = new Date(date + ' ' + time).getTime()
      let hash = hashCode(messageBlock)
      if(typeof chatCache[cellId] == 'undefined'
        || chatCache[cellId].hash != hash
        || typeof chatCache[cellId].emotions == 'undefined'
        || typeof chatCache[cellId].category != 'string'
        || typeof chatCache[cellId].date == 'undefined'
        || !chatCache[cellId].date
        || chatCache[cellId].summary.match(/Find the derivative/gi)
        || chatCache[cellId].category == '<h1>'
      ) {
        let summary = await askLlamaAboutConversation(currentMessages)
        let emotions = await askLlamaAboutEmotions(currentMessages)
        let category = await askLlamaAboutCategory(currentMessages)
        storeChatHistory(cellId, mtime, summary, emotions, category, hash, now)
      }
      currentMessages = []
      currentTotal = 0
      cellCounter++
      cellId = chatPath + '[' + cellCounter + ']'
    }

    currentMessages[currentMessages.length] = chatHistory[i]
    currentTotal += chatHistory[i].length
  }
  if(currentMessages.length > 0) {
    // process now
    let messageBlock = currentMessages.join('\n')
    let date = CHAT_DATES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
    let time = CHAT_TIMES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
    let now = new Date(date + ' ' + time).getTime()
    let hash = hashCode(messageBlock)
    if(typeof chatCache[cellId] == 'undefined'
      || chatCache[cellId].hash != hash
      || typeof chatCache[cellId].emotions == 'undefined'
      || typeof chatCache[cellId].category != 'string'
      || typeof chatCache[cellId].date == 'undefined'
    ) {
      let summary = await askLlamaAboutConversation(currentMessages)
      let emotions = await askLlamaAboutEmotions(currentMessages)
      let category = await askLlamaAboutCategory(currentMessages)
      storeChatHistory(cellId, mtime, summary, emotions, category, hash, now)
    }
    // TODO: store result
  }

  
}

module.exports = {
  askLlamaAboutChatLog,
  askLlamaAboutConversation
}



ask llm about emotions?



In [ ]:
const llama = importer.import('create llm session')

const EMOTIONS = [
  'Joyful',
  'Romantic',
  'Angry',
  'Confused',
  'Supportive',
  'Excited',
  'Nostalgic',
  'Grateful',
  'Sad',
  'Humorous',
  'Anxious',
  'Curious',
  'Inspired',
  'Defensive',
  'Assertive',
  'Empathetic',
  'Reflective',
  'Playful',
  'Hopeful',
  'Apologetic',
  'Lonely',
  'Proud',
  'Vulnerable',
  'Determined',
  'Aroused',
  'Neutral',
  'Dissident',
  'Rebelious',
  'Frustrated',
  'Helpful'
]

const EMOTION_HEX = [
  '#FFFF00',
  '#FFC0CB',
  '#FF0000',
  '#D8BFD8',
  '#90EE90',
  '#FFA500',
  '#4682B4',
  '#FFD700',
  '#00008B',
  '#FFFF99',
  '#800080',
  '#40E0D0',
  '#FFC107',
  '#800000',
  '#FF4500',
  '#E6E6FA',
  '#B0C4DE',
  '#00FFFF',
  '#FFFACD',
  '#B0E0E6',
  '#191970',
  '#FFD700',
  '#FFE4E1',
  '#DC143C'
]

async function askLlamaAboutEmotions(currentMessages, session) {
  const {llmPrompt} = (await llama)
  if(typeof currentMessages == 'string') {
    currentMessages = [currentMessages]
  }

  let q1 = 'Can you derive the emotional contexts of this conversation:\n' + 
  currentMessages.join('\n') + '\nOnly give the emotion in the response.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);

  let q2 = 'Based on this description which emotion best fits:\n' + 
    a1 + '\nOut of this list of emotions which one is the closest match:' +
    EMOTIONS.join(', ') + '\nOnly respond with the matching emotion.'
  console.log("User: " + q2);
  const a2 = await llmPrompt(q2);
  console.log("AI: " + a2);
  let emotions = a2.trim().split(/\n|,\s*|\s*- |\s*\* /gi)

  console.log(emotions)
  return EMOTIONS.filter(e => emotions.includes(e))[0]
}

module.exports = {
  askLlamaAboutEmotions,
  EMOTIONS,
  EMOTION_HEX
}



how to cache chat logs?



In [ ]:
const { chatCache } = importer.import('cache chat history with llm descriptions')
const { updateCode } = importer.import('update code cell')

function storeChatHistory (cellId, mtime, summary, emotions, category, hash, date) {
  chatCache[cellId] = {
    mtime,
    summary,
    emotions,
    category,
    hash,
    date
  }
  var code = `
// cell cache automatically replaced
var chatCache = ${JSON.stringify(chatCache, null, 4)}

module.exports = {
  chatCache
}
`
  var cacheCell = importer.interpret('cache chat history with llm descriptions')
  updateCode(cacheCell, code)
}

module.exports = {
  storeChatHistory
}



ask llm about chat conversations?


In [ ]:
const llama = importer.import('create llm session')

async function askLlamaAboutConversation(currentMessages) {
  const {llmPrompt} = (await llama)
  let q1 = 'Can you summarize in two sentences what this conversation is about:\n' + 
  currentMessages.join('\n') + '\nPlease discard any pleasantries, documentation only.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);
  return a1.trim()
}

async function askLlamaAboutCategory(currentMessages) {
  const {llmPrompt} = (await llama)
  let q1 = 'Categorize this conversation in two or three words:\n' + 
  currentMessages.join('\n') + '\nOnly respond with the category.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);
  return a1.trim().split(/\n|,\s*|\s*- |\s*\* /gi)[0]
}

module.exports = {
  askLlamaAboutConversation,
  askLlamaAboutCategory
}




how to scan code history?







how to scan calendar journal entries?

how to ask if my theories are correct?


